In [3]:
import requests
from bs4 import BeautifulSoup
import csv

# URL of the main page
main_url = 'https://results.eci.gov.in/AcResultByeJune2024/index.htm'

# Fetch the main page
response = requests.get(main_url)
webpage_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(webpage_content, 'html.parser')

# Find all boxes and extract the links
boxes = soup.find_all('div', class_='col-md-3 col-12')

# List to hold all the extracted data
data = [["S.N.", "Constituency", "Candidate", "Party", "EVM Votes", "Postal Votes", "Total Votes", "% of Votes"]]

# Loop through all boxes
for box in boxes:
    # Get the constituency name from the <h3> tag
    constituency_name = box.find('h3').text.strip()
    
    # Get the link for each box
    box_link = box.find('a')['href']
    box_url = 'https://results.eci.gov.in/AcResultByeJune2024/' + box_link
    
    # Fetch the page of the box
    response_box = requests.get(box_url)
    box_content = response_box.content
    
    # Parse the HTML content of the box page
    soup_box = BeautifulSoup(box_content, 'html.parser')
    
    # Locate the 'Constituency Wise Table View' link in the HTML
    table_view_link_tag = soup_box.select_one('a[href*="Constituencywise"]')
    if table_view_link_tag:
        table_view_link = table_view_link_tag['href']
        table_view_url = 'https://results.eci.gov.in/AcResultByeJune2024/' + table_view_link
        
        # Fetch the 'Constituency Wise Table View' page
        response_table_view = requests.get(table_view_url)
        table_view_content = response_table_view.content
        
        # Parse the HTML content of the table view page
        soup_table_view = BeautifulSoup(table_view_content, 'html.parser')
        
        # Extract the desired data from the table body
        tbody = soup_table_view.find('tbody')
        
        if tbody:
            rows = tbody.find_all('tr')
            
            # Loop through all rows and extract data
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 7:  # Ensure there are 7 columns as expected
                    sn = cols[0].text.strip()
                    candidate = cols[1].text.strip()
                    party = cols[2].text.strip()
                    evm_votes = cols[3].text.strip()
                    postal_votes = cols[4].text.strip()
                    total_votes = cols[5].text.strip()
                    percent_votes = cols[6].text.strip()
                    
                    data.append([sn, constituency_name, candidate, party, evm_votes, postal_votes, total_votes, percent_votes])
        else:
            print(f"Table body (tbody) not found for {table_view_url}!")
    else:
        print(f"'Constituency Wise Table View' link not found for {box_url}!")

# Write the data into a CSV file
with open('all_boxes_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("Data has been written to all_boxes_results.csv")

Data has been written to all_boxes_results.csv
